In [ ]:
# now that I have my data cleaned, let's transform it to a format that mistral understands

the desired format : 
{
  "messages": [
    {
      "role": "user",
      "content": "User interaction n°1 contained in document n°1"
    },
    {
      "role": "assistant",
      "content": "Bot interaction n°1 contained in document n°1"
    },
  ]
}
{
  "messages": [
    {
      "role": "user",
      "content": "User interaction n°1 contained in document n°1"
    },
    {
      "role": "assistant",
      "content": "Bot interaction n°1 contained in document n°1"
    },
  ]
}

In [ ]:
import pandas as pd
import json


# Function to transform DataFrame to JSONL format
def dataframe_to_jsonl(df, output_file):
    with open(output_file, "w") as f:
        for _, row in df.iterrows():
            messages = [
                {"role": "user", "content": row["question"]},
                {"role": "assistant", "content": row["answer"]},
            ]
            json_record = {"messages": messages}
            f.write(json.dumps(json_record) + "\n")


def correct_french_special_char(text):

    text = text.replace("\u00e9", "e")
    text = text.replace("\u2019", "'")
    text = text.replace("\u00e0", "a")
    text = text.replace("\u00fb", "u")
    text = text.replace("\u00e8", "e")
    text = text.replace("\u00f9", "u")
    text = text.replace("\u00ea", "e")
    text = text.replace("\u00e7", "c")
    text = text.replace("\u00ca", "e")
    text = text.replace("\u00c9", "e")
    text = text.replace("\u00ee", "i")
    text = text.replace("\u00f4", "o")
    text = text.replace("\u00e2", "a")
    text = text.replace("\u00c0", "a")
    text = text.replace("\u00a7", "")
    text = text.replace("\u00a0", "")
    text = text.replace("\u00ab", "")
    text = text.replace("\u00bb", "")
    text = text.replace("\u00b2", "2")
    text = text.replace("\u00d7", "x")
    text = text.replace("\u00f7", "")
    return text

In [ ]:
df = pd.read_csv("output.csv")

In [ ]:
# DroitQA, TrainQA done
df = pd.read_csv("output.csv")
df = df.rename(columns={"Question": "question", "Answer": "answer"})
df["question"] = df["question"].apply(lambda x: correct_french_special_char(x))
df["answer"] = df["answer"].apply(lambda x: correct_french_special_char(x))

# Specify the output file path
output_file = "output.jsonl"

# Transform the DataFrame and save to JSONL
dataframe_to_jsonl(df, output_file)

In [ ]:
# questions are done, let's do the text, then fine tuning!
# split the legis_FR cause tooo big

import re
import json

# Read the content of the file
file_path = "../Data/Legis_FR.txt"
with open(file_path, "r", encoding="utf-8") as file:
    content = file.read()


content = content.replace("\n", " ")
content = content.lower()
# Define a regular expression pattern to match articles
article_pattern = re.compile(
    r"art\.\s*\d{1,4}\s*.*?(?=art\.\s*\d{1,4}\s*|\Z)", re.IGNORECASE
)

# Find all articles using the regular expression
articles = article_pattern.findall(content)

# filters on the ones who have a len that is too small, cause they are not real articles
art = pd.DataFrame(articles)
art = art.rename(columns={0: "text"})
art["len"] = art.text.apply(lambda x: len(x))

articles = list(art[art.len > 300]["text"])


# Prepare the JSONL content
jsonl_content = []
for article in articles:
    article_text = " ".join(article.split())
    jsonl_content.append({"text": article_text})

# Save to JSONL file
output_path = "../Data/Legis_FR_articles.jsonl"
with open(output_path, "w", encoding="utf-8") as jsonl_file:
    for entry in jsonl_content:
        jsonl_file.write(json.dumps(entry, ensure_ascii=False) + "\n")

output_path

In [ ]:
# questions are done, let's do the text, then fine tuning!
# split the legis_FR cause tooo big

import re
import json

# Read the content of the file
file_path = "../Data/Legis_dutch.txt"
with open(file_path, "r", encoding="utf-8") as file:
    content = file.read()


content = content.replace("\n", " ")
content = content.lower()
# Define a regular expression pattern to match articles
article_pattern = re.compile(
    r"art\.\s*\d{1,4}\s*.*?(?=art\.\s*\d{1,4}\s*|\Z)", re.IGNORECASE
)

In [ ]:
# Find all articles using the regular expression
articles = article_pattern.findall(content)

# filters on the ones who have a len that is too small, cause they are not real articles
art = pd.DataFrame(articles)
art = art.rename(columns={0: "text"})
art["len"] = art.text.apply(lambda x: len(x))

articles = list(art[art.len > 300]["text"])


# Prepare the JSONL content
jsonl_content = []
for article in articles:
    article_text = " ".join(article.split())
    jsonl_content.append({"text": article_text})

# Save to JSONL file
output_path = "../Data/Legis_NL_articles.jsonl"
with open(output_path, "w", encoding="utf-8") as jsonl_file:
    for entry in jsonl_content:
        jsonl_file.write(json.dumps(entry, ensure_ascii=False) + "\n")

output_path